In [11]:
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.models import xDeepFM
from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

from deepctr_torch.callbacks import EarlyStopping, ModelCheckpoint

import torch

In [19]:
a = pd.read_csv('./output/96499_xDeepFM_4_Batch256_bins5_only_sparse.csv', sep='\t')

In [20]:
b = pd.read_csv('./output/3957_xDeepFM_4_Batch256_bins10_only_sparse.csv', sep='\t')

In [22]:
a['is_installed'] = (a['is_installed'] + b['is_installed']) / 2
a['is_clicked'] = (a['is_clicked'] + b['is_clicked']) / 2

In [24]:
a.to_csv('./output/avg_96499_3957.csv', index=False, sep='\t')

In [25]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import os
import re
import math
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from lightgbm import LGBMRegressor, LGBMClassifier
from xgboost import XGBRegressor, XGBClassifier
from catboost import CatBoostRegressor, CatBoostClassifier
import lightgbm as lgb
import xgboost as xgb
import catboost as cab

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold, GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation, NMF, TruncatedSVD
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor, GradientBoostingRegressor, HistGradientBoostingRegressor, StackingRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, StackingClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from sklearn.preprocessing import KBinsDiscretizer
import matplotlib.pyplot as plt

import warnings

warnings.filterwarnings('ignore')

读入

In [26]:
test_data = pd.read_csv('data/test/000000000000.csv', sep='\t')
test_data.shape 

(160973, 80)

In [27]:
train_data = pd.read_csv('./data/train.csv', sep='\t')
train_data.shape 

(3485852, 82)

In [28]:
train_data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79,is_clicked,is_installed
0,2541188,57,26325,22294,9018,25604,943,27941,19203,21533,...,2.855607,2.284486,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0
1,2541440,60,5156,22294,18971,21545,3448,27941,19606,14659,...,0.000000,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,0,0
2,2541480,65,30256,22294,11104,21545,20366,27941,19203,31372,...,0.571121,0.000000,0.000000,1.156922,0.269948,0.0,0.0,0.0,1,0
3,2541780,63,17216,7152,15742,21545,23877,27941,19606,869,...,0.000000,0.000000,0.000000,0.347077,0.000000,0.0,0.0,0.0,0,0
4,2541833,60,9317,22294,26866,21545,32370,27941,21218,14659,...,0.000000,0.000000,0.115692,1.156922,0.269948,0.0,0.0,0.0,0,0


In [29]:
test_data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
0,64505,67,26325,7152,21563,19475,31440,27941,21621,14659,...,1.519085,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0
1,64506,67,20095,563,22861,19475,21280,27941,19203,14659,...,0.000000,0.0,0.000000,0.0,0.115692,1.156922,0.269948,0.0,0.0,0.0
2,64507,67,890,22294,18294,21545,20210,27941,18800,9638,...,0.982995,0.0,0.000000,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
3,64508,67,20095,563,22861,25604,21280,27941,18800,14659,...,0.000000,0.0,2.284486,0.0,0.000000,1.156922,0.269948,0.0,0.0,0.0
4,64509,67,27426,22294,11338,19475,23855,27941,21218,9638,...,0.986040,0.0,0.000000,0.0,0.077128,0.077128,0.077128,0.0,0.0,0.0


In [30]:
data = pd.concat([train_data, test_data])

In [31]:
cat_features = ['f_{}'.format(i) for i in range(1, 42)]
bin_features = ['f_{}'.format(i) for i in range(33, 42)]
num_features = ['f_{}'.format(i) for i in range(42, 80)]
date_features = ['f_1']

In [32]:
sparse_features = cat_features
dense_features = num_features

In [33]:
data[dense_features].describe()

,f_42,f_43,f_44,f_45,f_46,f_47,f_48,f_49,f_50,f_51,...,f_70,f_71,f_72,f_73,f_74,f_75,f_76,f_77,f_78,f_79
count,3.646825e+06,3.455631e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.455631e+06,...,3.455631e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06,3.646825e+06
mean,1.887600e+01,1.084403e+00,7.365268e-03,9.394906e-03,3.266683e-03,1.258501e-02,2.191617e-02,1.214932e-02,3.404248e-02,9.676367e+00,...,1.041742e+00,4.324664e-02,8.554965e-01,2.504329e-01,6.511000e-02,1.002406e+00,2.249009e-01,6.870399e-02,1.075348e+00,3.605268e-01
std,4.260112e+01,9.153970e-01,8.579235e-02,9.854993e-02,5.213730e-02,1.203414e-01,1.397989e-01,9.934114e-02,1.820592e-01,8.557606e+00,...,1.094155e+00,1.840033e-01,1.288103e+00,5.601684e-01,5.480815e-02,3.558279e-01,9.442490e-02,1.626345e+00,6.895088e+00,3.814547e+00
min,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.166491e-06,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.930869e+00,7.527682e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2.472436e+00,...,1.559339e-04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.156922e+00,2.699485e-01,0.000000e+00,0.000000e+00,0.000000e+00
50%,1.037373e+01,1.157403e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,6.783773e+00,...,8.758587e-01,0.000000e+00,5.711215e-01,0.000000e+00,1.156922e-01,1.156922e+00,2.699485e-01,0.000000e+00,0.000000e+00,0.000000e+00
75%,2.441106e+01,1.739014e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.579169e+01,...,1.456695e+00,0.000000e+00,1.142243e+00,5.711215e-01,1.156922e-01,1.156922e+00,2.699485e-01,0.000000e+00,0.000000e+00,0.000000e+00
max,2.348182e+04,1.176025e+02,1.313579e+01,1.427804e+01,5.711215e+00,4.740308e+01,1.542028e+01,1.256467e+01,2.855607e+01,3.072614e+01,...,7.495700e+01,1.713364e+00,5.711215e+00,3.997850e+00,1.156922e-01,1.156922e+00,2.699485e-01,1.121537e+02,4.859995e+02,2.243075e+02


In [36]:
for feat in dense_features:
    print(feat, data[feat].max() - data[feat].min())

f_42 23481.81553772541
f_43 117.60245036343844
f_44 13.135793838855793
f_45 14.278036781364992
f_46 5.7112147125459956
f_47 47.403082114131784
f_48 15.42027972387419
f_49 12.564672367601194
f_50 28.556073562729978
f_51 30.726134762362477
f_52 9.371068634246576
f_53 5.0518929674333375
f_54 17.238138598799253
f_55 71.18927036551102
f_56 13.111783274254467
f_57 71.18927036551102
f_58 68.8755609646012
f_59 4523.565856375785
f_60 16570.973256982776
f_61 1513.4018051210676
f_62 1190.5917223050874
f_63 1843.5974706186632
f_64 415706830424.0
f_65 245375.66598079557
f_66 97.81664179648071
f_67 14.873527568031658
f_68 64.96885037542945
f_69 39.87544130370358
f_70 74.9569994558488
f_71 1.7133644137637989
f_72 5.711214712545996
f_73 3.997850298782197
f_74 0.11569220536106882
f_75 1.156922053610688
f_76 0.2699484791758273
f_77 112.15372537291115
f_78 485.9994766159483
f_79 224.3074507458223


In [10]:
dense_mean = data[dense_features].mean()

In [11]:
data[sparse_features] = data[sparse_features].fillna(-1, )
# data[dense_features] = data[dense_features].fillna(0, )
data[dense_features] = data[dense_features].fillna(dense_mean, )
target = ['is_installed']

数据处理

In [12]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [13]:
# TODO: 分桶

In [14]:
dense_bins = 10

In [15]:
add_sparse_features = []
for fea in tqdm(dense_features, total=len(dense_features)):
    discretizer = KBinsDiscretizer(n_bins=dense_bins, encode='ordinal', strategy='kmeans')  # 等频quantile，等宽uniform
    data[fea + '_encode'] = discretizer.fit_transform(np.array(data[fea].tolist()).reshape(-1, 1))
    add_sparse_features.append(fea + '_encode')

100%|██████████| 38/38 [03:44<00:00,  5.92s/it]


In [16]:
add_sparse_features = [fea + '_encode' for fea in dense_features]

In [17]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [18]:
# data = pd.read_csv('./data/data_bins_10.csv')

In [19]:
data.head()

,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,...,f_70_encode,f_71_encode,f_72_encode,f_73_encode,f_74_encode,f_75_encode,f_76_encode,f_77_encode,f_78_encode,f_79_encode
0,2541188,12,114,4,191,4,155,0,1,4,...,2.0,2.0,4.0,4.0,3.0,9.0,7.0,0.0,0.0,0.0
1,2541440,15,21,4,390,3,559,0,2,3,...,1.0,0.0,0.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
2,2541480,20,131,4,228,3,3261,0,1,6,...,3.0,0.0,1.0,0.0,0.0,9.0,7.0,0.0,0.0,0.0
3,2541780,18,73,2,317,3,3843,0,2,0,...,1.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0
4,2541833,15,44,4,519,3,5179,0,4,3,...,1.0,0.0,0.0,0.0,3.0,9.0,7.0,0.0,0.0,0.0


In [104]:
sparse_dim = 2
dense_dim = 1

In [105]:
# fixlen_feature_columns = \
#     [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=sparse_dim) for i, feat in enumerate(sparse_features)] + \
#     [SparseFeat(feat, vocabulary_size=dense_bins, embedding_dim=sparse_dim) for i, feat in enumerate(add_sparse_features)] + \
#     [DenseFeat(feat, dense_dim) for feat in dense_features]

In [106]:
fixlen_feature_columns = \
    [SparseFeat(feat, vocabulary_size=data[feat].nunique(), embedding_dim=sparse_dim) for i, feat in enumerate(sparse_features)] + \
    [SparseFeat(feat, vocabulary_size=dense_bins, embedding_dim=sparse_dim) for i, feat in enumerate(add_sparse_features)]

In [107]:
dnn_feature_columns = fixlen_feature_columns
linear_feature_columns = fixlen_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [108]:
fixlen_feature_columns

[SparseFeat(name='f_1', vocabulary_size=23, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_1', group_name='default_group'),
 SparseFeat(name='f_2', vocabulary_size=139, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_2', group_name='default_group'),
 SparseFeat(name='f_3', vocabulary_size=5, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_3', group_name='default_group'),
 SparseFeat(name='f_4', vocabulary_size=638, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_4', group_name='default_group'),
 SparseFeat(name='f_5', vocabulary_size=6, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_5', group_name='default_group'),
 SparseFeat(name='f_6', vocabulary_size=5234, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_6', group_name='default_group'),
 SparseFeat(name='f_7', vocabulary_size=1, embedding_dim=2, use_hash=False, dtype='int32', embedding_name='f_7', group_name='default_group')

单模训练

In [109]:
seed = 2023
device = 'cuda:2'
model_name = 'xDeepFM'
batch_size = 256
epochs = 50
patience = 10

In [110]:
train = data[~data['is_clicked'].isna()]
test = data[data['is_clicked'].isna()]

In [111]:
train_model_input = {name: train[name] for name in feature_names}
test_model_input = {name: test[name] for name in feature_names}

In [112]:
model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device, seed=seed)
model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])

In [113]:
model 

xDeepFM(
  (embedding_dict): ModuleDict(
    (f_1): Embedding(23, 2)
    (f_2): Embedding(139, 2)
    (f_3): Embedding(5, 2)
    (f_4): Embedding(638, 2)
    (f_5): Embedding(6, 2)
    (f_6): Embedding(5234, 2)
    (f_7): Embedding(1, 2)
    (f_8): Embedding(6, 2)
    (f_9): Embedding(7, 2)
    (f_10): Embedding(3, 2)
    (f_11): Embedding(24, 2)
    (f_12): Embedding(26, 2)
    (f_13): Embedding(331, 2)
    (f_14): Embedding(19, 2)
    (f_15): Embedding(5854, 2)
    (f_16): Embedding(12, 2)
    (f_17): Embedding(49, 2)
    (f_18): Embedding(924, 2)
    (f_19): Embedding(19, 2)
    (f_20): Embedding(57, 2)
    (f_21): Embedding(35, 2)
    (f_22): Embedding(26, 2)
    (f_23): Embedding(4, 2)
    (f_24): Embedding(4, 2)
    (f_25): Embedding(3, 2)
    (f_26): Embedding(2, 2)
    (f_27): Embedding(2, 2)
    (f_28): Embedding(2, 2)
    (f_29): Embedding(2, 2)
    (f_30): Embedding(3, 2)
    (f_31): Embedding(3, 2)
    (f_32): Embedding(4, 2)
    (f_33): Embedding(2, 2)
    (f_34): Embeddin

In [114]:
es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=patience, mode='min')
mdckpt = ModelCheckpoint(filepath='ckpt/{}_{}_Batch{}_bins{}_onlysparse.ckpt'.format(model_name, sparse_dim, batch_size, dense_bins), monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')
history = model.fit(train_model_input, train_data[target].values, batch_size=batch_size, epochs=epochs, verbose=1, validation_split=0.2, callbacks=[es,mdckpt])

cuda:2
Train on 2788681 samples, validate on 697171 samples, 10894 steps per epoch


10894it [10:59, 16.52it/s]


Epoch 1/50
684s - loss:  0.2735 - binary_crossentropy:  0.2735 - val_binary_crossentropy:  0.2659
Epoch 00001: val_binary_crossentropy improved from inf to 0.26593, saving model to ckpt/xDeepFM_2_Batch256_bins10_onlysparse.ckpt


10894it [10:37, 17.10it/s]


Epoch 2/50
661s - loss:  0.2620 - binary_crossentropy:  0.2620 - val_binary_crossentropy:  0.2613
Epoch 00002: val_binary_crossentropy improved from 0.26593 to 0.26131, saving model to ckpt/xDeepFM_2_Batch256_bins10_onlysparse.ckpt


...

Epoch 11/50
678s - loss:  0.2011 - binary_crossentropy:  0.2008 - val_binary_crossentropy:  0.3402
Epoch 00011: val_binary_crossentropy did not improve from 0.26096


10894it [11:06, 16.34it/s]


Epoch 12/50
690s - loss:  0.1918 - binary_crossentropy:  0.1914 - val_binary_crossentropy:  0.3597
Epoch 00012: val_binary_crossentropy did not improve from 0.26096


10894it [10:22, 17.51it/s]


Epoch 13/50
646s - loss:  0.1830 - binary_crossentropy:  0.1826 - val_binary_crossentropy:  0.3827
Epoch 00013: val_binary_crossentropy did not improve from 0.26096
Epoch 00013: early stopping


单模测试

In [115]:
model = torch.load('ckpt/{}_{}_Batch{}_bins{}_onlysparse.ckpt'.format(model_name, sparse_dim, batch_size, dense_bins))

In [116]:
pred_ans = model.predict(test_model_input, batch_size=batch_size)

In [117]:
pred_ans.min(), pred_ans.max()

(7.910628596619063e-07, 0.9862615466117859)

In [118]:
pred_oof = model.predict(train_model_input, batch_size=batch_size)

In [119]:
logloss = metrics.log_loss(train_data[target], pred_oof)
acc = metrics.roc_auc_score(train_data[target], pred_oof)
precision = metrics.precision_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])
recall = metrics.recall_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])
f1 = metrics.f1_score(train_data[target], [1 if i >= 0.5 else 0 for i in pred_oof])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2549, AUC: 0.9146, Precision: 0.7872, Recall: 0.5268, F1 Score: 0.6312


In [120]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = pred_ans
submission.to_csv('./output/{}_{}_{}_{}_{}_Batch{}_bins{}_onlysparse.csv'.format(model_name, seed, len(feature_names), sparse_dim, dense_dim, batch_size, dense_bins), index=False, sep='\t')

In [121]:
# submission = submission.sort_values(by='RowId', axis=0, ascending=True)

In [122]:
# submission.to_csv('./output/{}_{}_{}_{}_{}_Batch{}_bins{}_onlysparse_ascending.csv'.format(model_name, seed, len(feature_names), sparse_dim, dense_dim, batch_size, dense_bins), index=False, sep='\t')

交叉验证

In [48]:
K = 5
seed = 2023
device = 'cuda:3'
model_name = 'xDeepFM'
batch_size = 2048
epochs = 50
patience = 10

In [49]:
train = data[~data['is_installed'].isna()]
test = data[data['is_installed'].isna()]
label = train['is_installed'].astype('int')

In [50]:
KF = StratifiedKFold(n_splits=K, random_state=seed, shuffle=True)
oof_ins = np.zeros(len(train))
predictions_ins = np.zeros((len(test)))
print(len(feature_names))

79


In [55]:
for fold_, (trn_idx, val_idx) in enumerate(KF.split(train.values, label.values)):
    print("fold n°{}".format(fold_))
    
    print(model_name, len(trn_idx), len(val_idx), len(feature_names))
    train_fold = train.iloc[trn_idx]
    train_model_input = {name: train_fold[name] for name in feature_names}
    train_model_label = label.iloc[trn_idx].values
    val_fold = train.iloc[val_idx]
    val_model_input = {name: val_fold[name] for name in feature_names}
    val_model_label = label.iloc[val_idx].values
    test_model_input = {name: test[name] for name in feature_names}

    model = xDeepFM(linear_feature_columns, dnn_feature_columns, task='binary', device=device, seed=seed)
    model.compile("adam", "binary_crossentropy", metrics=['binary_crossentropy'])
    
    es = EarlyStopping(monitor='val_binary_crossentropy', min_delta=0, verbose=1, patience=patience, mode='min')
    mdckpt = ModelCheckpoint(filepath='ckpt/{}_{}_fold{}.ckpt'.format(model_name, sparse_dim, fold_), 
                             monitor='val_binary_crossentropy', verbose=1, save_best_only=True, mode='min')
    history = model.fit(train_model_input, train_model_label, batch_size=batch_size, epochs=epochs, verbose=1, 
                        validation_data=(val_model_input, val_model_label), callbacks=[es, mdckpt])

    model = torch.load('ckpt/{}_{}_fold{}.ckpt'.format(model_name, sparse_dim, fold_))
    oof_ins[val_idx] = model.predict(val_model_input, batch_size=batch_size).flatten()
    predictions_ins[:] += model.predict(test_model_input, batch_size=batch_size).flatten() / K

fold n°0
xDeepFM 2788681 697171 79
cuda:2
Train on 2788681 samples, validate on 697171 samples, 1362 steps per epoch


100%|██████████| 1362/1362 [02:15<00:00, 10.08it/s]


Epoch 1/50
147s - loss:  0.2862 - binary_crossentropy:  0.2862 - val_binary_crossentropy:  0.2723
Epoch 00001: val_binary_crossentropy improved from inf to 0.27231, saving model to ckpt/xDeepFM_16_fold0.ckpt


100%|██████████| 1362/1362 [02:12<00:00, 10.31it/s]


Epoch 2/50
143s - loss:  0.2658 - binary_crossentropy:  0.2658 - val_binary_crossentropy:  0.2614
Epoch 00002: val_binary_crossentropy improved from 0.27231 to 0.26136, saving model to ckpt/xDeepFM_16_fold0.ckpt


...

Epoch 13/50
139s - loss:  0.2323 - binary_crossentropy:  0.2322 - val_binary_crossentropy:  0.2755
Epoch 00013: val_binary_crossentropy did not improve from 0.25710


100%|██████████| 1362/1362 [02:05<00:00, 10.81it/s]


Epoch 14/50
137s - loss:  0.2297 - binary_crossentropy:  0.2296 - val_binary_crossentropy:  0.2784
Epoch 00014: val_binary_crossentropy did not improve from 0.25710


100%|██████████| 1362/1362 [02:09<00:00, 10.54it/s]


Epoch 15/50
140s - loss:  0.2268 - binary_crossentropy:  0.2267 - val_binary_crossentropy:  0.2839
Epoch 00015: val_binary_crossentropy did not improve from 0.25710
Epoch 00015: early stopping


In [56]:
logloss = metrics.log_loss(label, oof_ins)
acc = metrics.roc_auc_score(label, oof_ins)
precision = metrics.precision_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])
recall = metrics.recall_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])
f1 = metrics.f1_score(label, [1 if i >= 0.5 else 0 for i in oof_ins])

print(f"Logloss: {logloss:.4f}, AUC: {acc:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}")

Logloss: 0.2582, AUC: 0.9118, Precision: 0.7882, Recall: 0.5204, F1 Score: 0.6269


In [57]:
predictions_ins.min(), predictions_ins.max()

(6.823736111982726e-06, 0.9711936712265015)

In [58]:
submission = pd.DataFrame()
submission["RowId"] = test_data["f_0"]
submission["is_clicked"] = np.random.random((test_data.shape[0]))
submission["is_installed"] = predictions_ins
submission.to_csv('./output/{}_{}_{}_{}_{}_Kfold{}.csv'.format(model_name, seed, len(feature_names), sparse_dim, dense_dim, K), index=False, sep='\t')

111